In [1]:
import pywikibot
import re

In [2]:
site = pywikibot.Site('fr', 'wiktionary')
print(site)

wiktionary:fr


In [3]:
# This parameter represents the character or set of consecutive characters 
# to be replaced
old_symbol = 'g' # regular 'g' letter 

# This parameter represents the character or set of consecutive characters 
# that will replace the old_symbol
new_symbol = 'ɡ' # voice velar stop (U+0261)

VERBOSE = False    

In [4]:
words = ['segmental']

In [5]:
def replace_pronunciations(text, pronunciations, old_symbol, new_symbol, verbose=False):
    nb_replacements = 0
    for pron in pronunciations:
        if verbose:
            print('found:', pron)
        old_string = pron[0] + pron[1] + pron[2]
        pron_new = pron[1].replace(old_symbol, new_symbol)
        new_string = pron[0] + pron_new + pron[2]
        print('replacing %s with %s' % (old_string, new_string))    
        text = text.replace(old_string, new_string)
        nb_replacements += 1
    print('nb_replacements:%d', nb_replacements)
    return text

In [6]:
def process_words(words, commit=False, verbose=False):
    for word in words:
        page = pywikibot.Page(site, word)
        if not page.exists():
            print('page %s does not exists' % word)
            continue
        print('page=%s' % word)
        #print(page.getVersionHistoryTable())
        page_text = page.text
        if verbose:
            print('#########################')
            print('old page text:', page_text)
            print('#########################')
            
        # Look for the pronunciation within the "ligne de forme"
        pron_head = '{{pron'
        pron_foot = 'fr}}'
        pronunciations = re.findall('('+ pron_head + '\|)'+ '([^\|\}]{1,})' + '(\|'+pron_foot+')', page_text)
        page_text = replace_pronunciations(page_text, pronunciations, old_symbol, new_symbol, verbose)

        # Look for the 2nd pronunciation
        pron_hd = '{{fr'
        pron_ft = '}}'
        pronunciations = re.findall('('+pron_hd + '[^\}]{0,}' + '\|)'+ '([^\|\}]{1,})' + '([^\}]{0,}'+pron_ft+')', page_text)
        page_text = replace_pronunciations(page_text, pronunciations, old_symbol, new_symbol, verbose)
        
        if verbose:
            print('#########################')
            print('new page text:', page_text)
            print('#########################')
            
        # string to summarize the work done in the edit comment
        summary = "prononciation: remplacement de /"+ old_symbol + "/ par /"+ new_symbol +"/."
        print(summary)
        
        page.text = page_text
        if commit:
            print('committing')
            page.save(summary=summary, botflag=True, quiet=False)

In [7]:
process_words(words, commit=True, VERBOSE=False)

page=segmental
replacing {{pron|sɛg.mɑ̃.tal|fr}} with {{pron|sɛɡ.mɑ̃.tal|fr}}
nb_replacements:%d 1
replacing {{fr-accord-al|segment|sɛg.mɑ̃.t}} with {{fr-accord-al|segment|sɛɡ.mɑ̃.t}}
nb_replacements:%d 1
prononciation: remplacement de /g/ par /ɡ/.
committing


Sleeping for 9.4 seconds, 2020-01-03 13:45:02
Page [[segmental]] saved
